In [1]:
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import pickle

In [2]:
# 读取采样数据对应的卫星数据
# 采样数据45524条，这里读取的是相对应的45524条卫星数据
grid,long_name = {},{}
path = "../0 save data/satellite grid data/"

# 读取，保存到grid中
files = os.listdir(path)
for file in files:
    with open(path+file,'rb') as f:
        name = file.split(".")[0]
        short_name =name.split("&")[1]
        long_name[short_name] =  name.split("&")[0]
        grid[short_name] = pickle.load(f)

# 匹配的数据是1*3*3的，也就是一天的时间窗口和3*3的空间窗口
cv = {}
N,C,L1,L2 = grid["sst"].shape

# 对时间窗口和空间窗口进行处理
for var in grid:
    print(var,grid[var].shape,end="\t")
    
    # 展平空间3*3的网格
    flatten = grid[var].reshape(N*C,L1*L2)
    print(flatten.shape,end="\t")
    
    # 求3*3网格的平均值
    nan_mean = np.nanmean(flatten,axis=1)
    print(nan_mean.shape,end="\t")
    
    # vailable data >= 5，有效值数量大于5个
    use = np.isnan(flatten).sum(axis=1)<5
    
    # 这里计算变异系数并保存，由于变异系数是只针对Rrs的，因此在下一步中使用
    if var in ["Rrs_412","Rrs_443","Rrs_469","Rrs_488","Rrs_531","Rrs_547","Rrs_555"]: 
        cv[var] = np.nanstd(flatten,axis=1)/np.nanmean(flatten,axis=1)
        
    
    # 有效值大于5个的网络平均值被保存
    nan_mean[~use] = np.nan
    grid[var] = nan_mean.reshape(N)
    print(grid[var].shape)
    

adg_443_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
adg_s_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
adg_unc_443_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
aph_443_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
aph_unc_443_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
chlor_a (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Zeu_lee (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
bbp_s_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
bbp_unc_443_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
bbp_443_giop (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
par (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_412 (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_443 (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_469 (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_488 (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_531 (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_547 (45524, 1, 3, 3)	(45524, 9)	(45524,)	(45524,)
Rrs_555 (45524, 1, 3, 3)	(45524, 9)	(4

C:\Users\yy\AppData\Local\Temp\ipykernel_13932\3476313104.py:22: RuntimeWarning: Mean of empty slice
  nan_mean = np.nanmean(flatten,axis=1)
C:\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\yy\AppData\Local\Temp\ipykernel_13932\3476313104.py:30: RuntimeWarning: Mean of empty slice
  cv[var] = np.nanstd(flatten,axis=1)/np.nanmean(flatten,axis=1)


In [3]:
# 这一步处理变异系数，只有Rrs变异系数小于0.15的数据才会使用，删除变异系数大于等于0.15的数据
cv1 = np.array(list(cv.values()))
del_cv = np.nanmedian(cv1,axis=0)>=0.15
for var in grid:
    grid[var][del_cv] = np.nan

C:\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [4]:
# case1 的限制
# case1 = (grid["Rrs_412"]/grid["Rrs_443"]>=1.0)
case1 = ( (grid["a_443_giop"]-grid["adg_443_giop"])/grid["a_443_giop"]>=0.5 )
for var in grid:
    grid[var][~case1] = np.nan

In [5]:
# case1 的严格限制，做测试时使用的，这里不用，注释了
# # RR12 = grid["Rrs_412"]/grid["Rrs_443"]
# RR53 = grid["Rrs_555"]/grid["Rrs_488"]

# # RR12_cs1 = 0.9351+0.113/RR53-0.0217/(RR53)**2+0.003/(RR53)**2
# Rrs555_cs1 = 0.0006+0.0027*RR53-0.0004*(RR53)**2-0.0002*(RR53)**3
# nu = 0.5
# strict_case1 = ((1-nu)*Rrs555_cs1<grid["Rrs_555"]) & ((1+nu)*Rrs555_cs1>grid["Rrs_555"])
# for var in grid:
#     grid[var][~strict_case1] = np.nan

In [6]:
# 由于选择输入卫星数据的不同，会产生不一样的数据量。这里控制相同的数据，保证相同的数据量
use_satellite_variables = ['adg_443_giop','adg_s_giop','bb_412_giop','bb_443_giop','bb_469_giop','bb_488_giop','bb_531_giop','bb_547_giop','bb_555_giop','bb_645_giop','bb_667_giop','bb_678_giop','a_412_giop','a_443_giop','a_469_giop','a_488_giop','a_531_giop','a_547_giop','a_555_giop','a_645_giop','a_667_giop','a_678_giop','Rrs_412','Rrs_443','Rrs_469','Rrs_488','Rrs_531','Rrs_547','Rrs_555','Rrs_645','Rrs_667','Rrs_678','par','Zeu_lee','chlor_a','sst']

nan_data = np.zeros(N,dtype=bool)
for var in use_satellite_variables:
    nan_data = nan_data|np.isnan(grid[var])
for var in grid:
    grid[var][nan_data] = np.nan

In [8]:
# 保存数据
with open("../0 save data/4 extract data.pkl", 'wb') as f:
    pickle.dump([grid, long_name, case1], f)